In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [1]:
!mamba install -c conda-forge rdkit


Looking for: ['rdkit']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.10.*
  - python 3.10.*
  - python_abi 3.10.* *cp310*
  - cudatoolkit *.*.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Draw

from IPython.display import display

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/Data

/content/drive/.shortcut-targets-by-id/1lU6QjJaBcElIvfuufmZL2sO12RkRuYUm/Data


In [5]:
ls

adata_excluded_ids.csv  condacolab_install.log  leash-BELKA.zip         multiome_var_meta.csv
adata_obs_meta.csv      de_train.parquet        multiome_obs_meta.csv   sample_submission.csv
adata_train.parquet     id_map.csv              multiome_train.parquet


## Importar bases


In [6]:
adata_excluded = pd.read_csv('adata_excluded_ids.csv')

In [7]:
adata_obs = pd.read_csv('adata_obs_meta.csv')

In [8]:
adata_train = pd.read_parquet('adata_train.parquet')

In [9]:
de_train = pd.read_parquet('de_train.parquet')

In [10]:
id_map = pd.read_csv('id_map.csv')

In [11]:
multiome_obs = pd.read_csv('multiome_obs_meta.csv')

In [12]:
multiome_train = pd.read_parquet('multiome_train.parquet')

In [13]:
multiome_var = pd.read_csv('multiome_var_meta.csv')

In [14]:
sample_submission = pd.read_csv('sample_submission.csv')